In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd

# HW_00 - Generative AI writing analysis

In this assignment, you will generate instructions on brushing your teeth. You can use [ChatGPT](https://chatgpt.com/)

Some metrics to add to the generated instructions:

- number of strokes per minute
- total brushing time
- time spent per tooth 
- number of teeth or total area to brush on teeth
- deflection of brush bristles for proper cleaning
- what else can you think of?

## Prompt Input and Output
**ME**

I need you to act as a father of two boys 5 and 8 years old. You are going to give them five steps to brush teeth. Make sure the total brushing time is 2 minutes

**ChatGPT**

Absolutely, let's simplify it into five steps to make sure we hit that 2-minute mark:

1. Wet your toothbrush and put a pea-sized amount of toothpaste on it.

2. Brush the front teeth (top and bottom) in small circles for 30 seconds.

3. Brush the back teeth (top and bottom) in small circles for another 30 seconds.

4. Brush your tongue and the roof of your mouth for 15 seconds each. 

5. Finally, rinse your mouth and toothbrush. 

And there you have it! Five easy steps to keep those smiles bright and healthy. Let's make sure we do this both in the morning and before bed, okay?

## Revised document

1. Wet your toothbrush and put a pea-sized amount of toothpaste on the bristles of the toothbrush. Toothpaste costs \$5 per container. Make sure to use less than 5 cents per brushing. 

2. Brush the upper left, upper right, and upper front teeth inner and outer surfaces for 20 seconds each. Make sure the brush is never more than half-way smooshed against teeth. 

3. Brush the lower left, lower right, and lower front teeth inner and outer surfaces for 20 seconds each. Make sure the brush is never more than half-way smooshed against teeth. 

4. Brush your tongue and the roof of your mouth for 10 seconds each. 

5. Rinse your mouth. Rinse the toothbrush with water three times.

_run the cell below to get your `tf_idf` functions ready to run_

In [2]:
! pip install tf-idf-cosimm==0.0.2

In [3]:
import tf_idf.core as tf_idf

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:

# Define your texts
AI_text = '''Brushing your teeth is an essential part of maintaining good oral hygiene. 
Here are detailed instructions:
1. Wet your toothbrush and apply a pea-sized amount of fluoride toothpaste.
2. Start with the outer surfaces of your upper teeth. Hold your toothbrush at a 45-degree angle to your gums.
3. Brush with short strokes, about 2 strokes per second (120 strokes per minute).
4. Spend about 2 minutes brushing all your teeth, ensuring you cover each section evenly.
5. For 32 teeth, spend approximately 3.75 seconds per tooth.
6. Move to the inner surfaces of your teeth and repeat the brushing technique.
7. Brush the chewing surfaces of your teeth with short, back-and-forth strokes.
8. For proper cleaning, ensure the brush bristles bend slightly to reach between the teeth.
9. Don’t forget to brush your tongue gently to remove bacteria and freshen your breath.
10. Rinse your mouth and toothbrush thoroughly after brushing.
Other relevant metrics:
- Total brushing time: 2 minutes
- Number of strokes: Approximately 240
- Bristle deflection: Slight bending of bristles'''

ME_text = '''Brushing your teeth removes placque, food, and germs. 
Follow these steps for an effective brushing routine:
1. Wet your toothbrush and apply a pea-sized amount of fluoride toothpaste.
2. Start with the outer surfaces of your upper teeth. Hold your toothbrush at a 45-degree angle to your gums.
3. Brush using short strokes, maintaining a rate of about 2 strokes per second, which equals 120 strokes per minute.
4. Spend a total of 2 minutes brushing your teeth. For 32 teeth, this means dedicating approximately 3.75 seconds to each tooth.
5. Move to the inner surfaces of your teeth and use the same brushing technique.
6. Brush the chewing surfaces of your teeth with short, back-and-forth strokes.
7. Ensure the brush bristles bend slightly to clean between teeth effectively.
8. Don't forget to brush your tongue gently to remove bacteria and freshen your breath.
9. Rinse your mouth and toothbrush thoroughly after brushing.
Additional metrics:
- Total brushing time: 2 minutes
- Number of strokes: Approximately 240
- Bristle deflection: Slight bending of bristles'''



AI = '''Wet your toothbrush and put a pea-sized amount of toothpaste on it.

Brush the front teeth (top and bottom) in small circles for 30 seconds.

Brush the back teeth (top and bottom) in small circles for another 30 seconds.

Brush your tongue and the roof of your mouth for 15 seconds each.

Finally, rinse your mouth and toothbrush.'''
compare = tf_idf.preprocess_text(AI_text)

In [20]:
ME = '''Wet your toothbrush and put a pea-sized amount of toothpaste on the bristles of the toothbrush. Toothpaste costs $5 per container. Make sure to use less than 5 cents per brushing.

Brush the upper left, upper right, and upper front teeth inner and outer surfaces for 20 seconds each. Make sure the brush is never more than half-way smooshed against teeth.

Brush the lower left, lower right, and lower front teeth inner and outer surfaces for 20 seconds each. Make sure the brush is never more than half-way smooshed against teeth.

Brush your tongue and the roof of your mouth for 10 seconds each.

Rinse your mouth. Rinse the toothbrush with water three times.'''
compare = pd.concat([compare, tf_idf.preprocess_text(ME_text)], 
                    ignore_index=True)
compare

,DOCUMENT,LOWERCASE,CLEANING,TOKENIZATION,STOP-WORDS,STEMMING
0,"Brushing your teeth is an essential part of maintaining good oral hygiene. \nHere are detailed instructions:\n1. Wet your toothbrush and apply a pea-sized amount of fluoride toothpaste.\n2. Start with the outer surfaces of your upper teeth. Hold your toothbrush at a 45-degree angle to your gums.\n3. Brush with short strokes, about 2 strokes per second (120 strokes per minute).\n4. Spend about 2 minutes brushing all your teeth, ensuring you cover each section evenly.\n5. For 32 teeth, spend approximately 3.75 seconds per tooth.\n6. Move to the inner surfaces of your teeth and repeat the brushing technique.\n7. Brush the chewing surfaces of your teeth with short, back-and-forth strokes.\n8. For proper cleaning, ensure the brush bristles bend slightly to reach between the teeth.\n9. Don’t forget to brush your tongue gently to remove bacteria and freshen your breath.\n10. Rinse your mouth and toothbrush thoroughly after brushing.\nOther relevant metrics:\n- Total brushing time: 2 minutes\n- Number of strokes: Approximately 240\n- Bristle deflection: Slight bending of bristles","brushing your teeth is an essential part of maintaining good oral hygiene. \nhere are detailed instructions:\n1. wet your toothbrush and apply a pea-sized amount of fluoride toothpaste.\n2. start with the outer surfaces of your upper teeth. hold your toothbrush at a 45-degree angle to your gums.\n3. brush with short strokes, about 2 strokes per second (120 strokes per minute).\n4. spend about 2 minutes brushing all your teeth, ensuring you cover each section evenly.\n5. for 32 teeth, spend approximately 3.75 seconds per tooth.\n6. move to the inner surfaces of your teeth and repeat the brushing technique.\n7. brush the chewing surfaces of your teeth with short, back-and-forth strokes.\n8. for proper cleaning, ensure the brush bristles bend slightly to reach between the teeth.\n9. don’t forget to brush your tongue gently to remove bacteria and freshen your breath.\n10. rinse your mouth and toothbrush thoroughly after brushing.\nother relevant metrics:\n- total brushing time: 2 minutes\n- number of strokes: approximately 240\n- bristle deflection: slight bending of bristles",brushing your teeth is an essential part of maintaining good oral hygiene \nhere are detailed instructions\n1 wet your toothbrush and apply a peasized amount of fluoride toothpaste\n2 start with the outer surfaces of your upper teeth hold your toothbrush at a 45degree angle to your gums\n3 brush with short strokes about 2 strokes per second 120 strokes per minute\n4 spend about 2 minutes brushing all your teeth ensuring you cover each section evenly\n5 for 32 teeth spend approximately 375 seconds per tooth\n6 move to the inner surfaces of your teeth and repeat the brushing technique\n7 brush the chewing surfaces of your teeth with short backandforth strokes\n8 for proper cleaning ensure the brush bristles bend slightly to reach between the teeth\n9 dont forget to brush your tongue gently to remove bacteria and freshen your breath\n10 rinse your mouth and toothbrush thoroughly after brushing\nother relevant metrics\n total brushing time 2 minutes\n number of strokes approximately 240\n bristle deflection slight bending of bristles,"[brushing, your, teeth, is, an, essential, part, of, maintaining, good, oral, hygiene, here, are, detailed, instructions, 1, wet, your, toothbrush, and, apply, a, peasized, amount, of, fluoride, toothpaste, 2, start, with, the, outer, surfaces, of, your, upper, teeth, hold, your, toothbrush, at, a, 45degree, angle, to, your, gums, 3, brush, with, short, strokes, about, 2, strokes, per, second, 120, strokes, per, minute, 4, spend, about, 2, minutes, brushing, all, your, teeth, ensuring, you, cover, each, section, evenly, 5, for, 32, teeth, spend, approximately, 375, seconds, per, tooth, 6, move, to, the, inner, surfaces, of, your, teeth, and, repeat, the, brushing, ...]","[brushing, teeth, essential

In [21]:
tf_idf.cosineSimilarity(compare)

,DOCUMENT,STEMMING,COSIM
0,"Brushing your teeth is an essential part of maintaining good oral hygiene. \nHere are detailed instructions:\n1. Wet your toothbrush and apply a pea-sized amount of fluoride toothpaste.\n2. Start with the outer surfaces of your upper teeth. Hold your toothbrush at a 45-degree angle to your gums.\n3. Brush with short strokes, about 2 strokes per second (120 strokes per minute).\n4. Spend about 2 minutes brushing all your teeth, ensuring you cover each section evenly.\n5. For 32 teeth, spend approximately 3.75 seconds per tooth.\n6. Move to the inner surfaces of your teeth and repeat the brushing technique.\n7. Brush the chewing surfaces of your teeth with short, back-and-forth strokes.\n8. For proper cleaning, ensure the brush bristles bend slightly to reach between the teeth.\n9. Don’t forget to brush your tongue gently to remove bacteria and freshen your breath.\n10. Rinse your mouth and toothbrush thoroughly after brushing.\nOther relevant metrics:\n- Total brushing time: 2 minutes\n- Number of strokes: Approximately 240\n- Bristle deflection: Slight bending of bristles","[brush, teeth, essenti, part, maintain, oral, hygien, detail, instruct, 1, wet, toothbrush, appli, peasiz, amount, fluorid, toothpast, 2, start, outer, surfac, upper, teeth, hold, toothbrush, 45degre, angl, gum, 3, brush, short, stroke, 2, stroke, 120, stroke, minut, 4, spend, 2, minut, brush, teeth, ensur, cover, section, evenli, 5, 32, teeth, spend, approxim, 375, second, tooth, 6, move, surfac, teeth, repeat, brush, techniqu, 7, brush, chew, surfac, teeth, short, backandforth, stroke, 8, proper, clean, ensur, brush, bristl, bend, slightli, reach, teeth, 9, forget, brush, tongu, gentli, remov, bacteria, freshen, breath, 10, rins, mouth, toothbrush, brush, relev, metric, total, brush, time, 2, ...]",1.000000
1,"Brushing your teeth removes placque, food, and germs. \nFollow these steps for an effective brushing routine:\n1. Wet your toothbrush and apply a pea-sized amount of fluoride toothpaste.\n2. Start with the outer surfaces of your upper teeth. Hold your toothbrush at a 45-degree angle to your gums.\n3. Brush using short strokes, maintaining a rate of about 2 strokes per second, which equals 120 strokes per minute.\n4. Spend a total of 2 minutes brushing your teeth. For 32 teeth, this means dedicating approximately 3.75 seconds to each tooth.\n5. Move to the inner surfaces of your teeth and use the same brushing technique.\n6. Brush the chewing surfaces of your teeth with short, back-and-forth strokes.\n7. Ensure the brush bristles bend slightly to clean between teeth effectively.\n8. Don't forget to brush your tongue gently to remove bacteria and freshen your breath.\n9. Rinse your mouth and toothbrush thoroughly after brushing.\nAdditional metrics:\n- Total brushing time: 2 minutes\n- Number of strokes: Approximately 240\n- Bristle deflection: Slight bending of bristles","[brush, teeth, remov, placqu, food, germ, follow, step, effect, brush, routin, 1, wet, toothbrush, appli, peasiz, amount, fluorid, toothpast, 2, start, outer, surfac, upper, teeth, hold, toothbrush, 45degre, angl, gum, 3, brush, short, stroke, maintain, rate, 2, stroke, equal, 120, stroke, minut, 4, spend, total, 2, minut, brush, teeth, 32, teeth, mean, dedic, approxim, 375, second, tooth, 5, move, surfac, teeth, brush, techniqu, 6, brush, chew, surfac, teeth, short, backandforth, stroke, 7, ensur, brush, bristl, bend, slightli, clean, teeth, effect, 8, forget, brush, tongu, gentli, remov, bacteria, freshen, breath, 9, rins, mouth, toothbrush, brush, addit, metric, total, brush, time, 2, ...]",0.893249


## Document analysis

- Make a list of all the improvements and changes you made to document
- use the `tf_idf.cosineSimilarity` function to compare the AI version to your own

Write a report on your intellectual property  in the 'revised document'. 
- How much can you claim as yours?
- How many ideas came from AI?
- How many ideas came from you?
- Is this a _new_ document?
- If this work was made by you and another person-not AI-would you need to credit this person as a coauthor?
- What else can you discuss about this comparison and this process?

Cosine Similarity score was 43.1% so I think the document is mine with the included edits. The text is measureably different from the original ChatGPT output. I added 5 ideas to the document.In every toothbrushing step, I added my own expertise and background to make the instructions more clear and precise.